In [1]:
pip install numpy librosa

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import numpy as np
import librosa

class ASVspoof19Loader:
    def __init__(self, audio_dir, label_file, sample_rate=16000):
        self.audio_dir   = audio_dir
        self.label_file  = label_file
        self.sample_rate = sample_rate
        self.label_map   = self.parsed_label_file()

    def parsed_label_file(self):
        label_dict = {}
        with open(self.label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                # ← use the 2nd column (parts[1]) for the actual audio filename
                file_id  = parts[1]
                filename = file_id + '.flac'
                label    = 0 if parts[-1]=='bonafide' else 1
                label_dict[filename] = label
        return label_dict

    def load_audio(self, filepath):
        try:
            wav, sr = librosa.load(filepath, sr=self.sample_rate)
            return wav / (np.max(np.abs(wav)) + 1e-9)
        except Exception as e:
            print(f"[Error] {filepath}: {e}")
            return None

    def load_dataset(self):
        data = []
        for fname, label in self.label_map.items():
            full_path = os.path.join(self.audio_dir, fname)
            if os.path.exists(full_path):
                audio = self.load_audio(full_path)
                if audio is not None:
                    data.append((audio, label))
            else:
                print(f"[Warning] Missing: {full_path}")
        return data

# ——— Test it ———
audio_folder  = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_train\flac"
protocol_file = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_cm_protocols\ASVspoof2019.LA.cm.train.trn.txt"

loader  = ASVspoof19Loader(audio_folder, protocol_file)
dataset = loader.load_dataset()

print("Total files loaded:", len(dataset))
if dataset:
    print("Sample waveform:", dataset[0][0][:10], "Label:", dataset[0][1])
else:
    print("No files loaded—check that your protocol and folder really match.")


[Warning] Missing: C:\ASVSpoof19\LA\ASVspoof2019_LA_train\flac\LA_T_9667455.flac
Total files loaded: 25379
Sample waveform: [0.00180054 0.00183105 0.00167847 0.00170898 0.00164795 0.00149536
 0.00146484 0.00149536 0.0015564  0.00146484] Label: 0


In [10]:
###OKR 3###
#Feature extraction
import librosa.display
import matplotlib.pyplot as plt

class FeatureExtractor:
    def __init__(self, dataset, sr = 16000):
        self.dataset = dataset
        self.sr = sr

    def extract_mfcc(self, y, n_mfcc = 13):
        return librosa.feature.mfcc(y=y, sr = self.sr, n_mfcc = n_mfcc)
    
    def extract_lpc(self, y, order = 16):
        #using a 25ms window
        frame = y[:int(self.sr * 0.025)]
        return librosa.lpc(y=frame, order=order)
    
    def extract_cqt(self, y):
        return librosa.cqt(y=y, sr=self.sr)
    
    def process_all(self):
        features = []
        for idx, (y, label) in enumerate(self.dataset):
            mfcc = self.extract_mfcc(y)
            cqt = self.extract_cqt(y)
            lpc = self.extract_lpc(y)
            features.append({
                'mfcc' : mfcc,
                'lpc' : lpc,
                'cqt' : cqt,
                'label' : label
            })
        if idx % 100 == 0:
            print(f"Processed {idx} files")
        if len(y) < 256:
            y = np.pad(y, (0,256 - len(y)), mode= 'constant')
        return features
    


audio_folder  = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_train\flac"
protocol_file = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_cm_protocols\ASVspoof2019.LA.cm.train.trn.txt"

# Run loader from OKR 1
loader = ASVspoof19Loader(audio_folder, protocol_file)
dataset = loader.load_dataset()

#runnign feature extraction
extractor = FeatureExtractor(dataset)
features = extractor.process_all()

#saving extracted features
np.save("features.npy", features)


KeyboardInterrupt: 